# Coursera 
## Applied data science capstone
### Week 4: Data definition

# PROBLEM DEFINITION
A Taxy company want to set up its business in Milano city, Italy. The business idea is to be affiliated with all the Hotels in the radius of 1km from the city center so as to exploit his smart mobility transport service. They want to offer the fastest pick up service to be as effective and fast as possible. For this reason they are looking for the perfect positioning for their headquartier. The rationale is to find the place that minimize either the average or the maximum distance from all Hotels in the very citycenter. Assuming an average speed of 40 km per hour to move in the city (https://www.statista.com/statistics/264703/average-speed-in-europes-15-most-congested-cities/) and neglecting all mobility and all congestion issues we want to define an estimation for a taxi service to reach every Hotel.

In [139]:
# Import package
import folium
# Define coordinates
Latitude = 45.465586
Longitude = 9.185944
# Create map of Milano 
MilanoMap = folium.Map(location=[Latitude, Longitude], zoom_start=14)
MilanoMap

# Data collection
We use Fouresquare data to get the cooridnates for every hotel, we look after the tag 'Hotel' in the query and in a radius of '1500' meters from Cordusio place, this distance approximate the restricted-traffic zone of the city. Notice that using a free developer account we retrive up to 100 places only so using the 'query' tag is particularly useful. Finally we can put all the Hotel on the map togheter with the circle defining the interested area.

In [138]:
# Set Foursquare API
CLIENT_ID = '02QBHIMZSD0O05JYQX4MBB0KOZR1KH3V2TQ34PV0CH1FKBEU' 
CLIENT_SECRET = 'NNGC5E03TOWAXLIWW1ERT03PH5T3YFFN1124KLOM31F5AE44'
VERSION = '20180605' 
LIMIT = 1000 
RADIUS = 1500
QUERY = 'Hotel'

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Latitude, 
    Longitude,  
    RADIUS, 
    LIMIT,
    QUERY
)
# get results
results = requests.get(url).json()
# Extract venues information
venues = results['response']['groups'][0]['items']
# flatten JSON format
venues = json_normalize(venues) 
# filter columns 
venues = venues.loc[:, ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']]
# filter the category for each row
venues['venue.categories'] = venues.apply(get_category_type, axis=1)
# clean columns
venues.columns = [col.split(".")[-1] for col in venues.columns]
# Define hotels dataframe
MilanoHotel = venues
# Print results
print('There are',MilanoHotel.shape[0],'Hotels in the area')
# Print results
MilanoHotel.head(10)

There are 95 Hotels in the area


<ipython-input-138-7b964d98b9d2>:37: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  venues = json_normalize(venues)


,name,categories,lat,lng
0,Park Hyatt Milan,Hotel,45.465532,9.188911
1,Room Mate Giulia Hotel,Hotel,45.465250,9.189396
2,BVLGARI Hotel Milano,Hotel,45.470149,9.189318
3,Mandarin Oriental,Hotel,45.469461,9.190876
4,Armani Hotel Milano,Hotel,45.470478,9.192882
5,Starhotels Rosa Grand,Hotel,45.464122,9.193692
6,Four Seasons Hotel Milano,Hotel,45.469372,9.195466
7,The Square,Hotel,45.461003,9.189338
8,HMS Hotel Milano Scala,Hotel,45.469061,9.186865
9,Hotel Spadari al Duomo,Hotel,45.463738,9.187130


In [140]:
for idx in MilanoHotel.index :
    folium.CircleMarker(
        [MilanoHotel.iloc[idx,2], MilanoHotel.iloc[idx,3]],
        radius=5,
        color = 'blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(MilanoMap)  

folium.Circle(
        [Latitude, Longitude],
        radius=RADIUS,
        color = 'red',
        fill=False,
        parse_html=False).add_to(MilanoMap) 

MilanoMap

In [104]:
MilanoHotel.iloc[idx,0]

'Heart Milan Appartments Asole'